In [2]:
from backend_manager import BackendManager
from backend_manager import Commands

#### 1) Connect to backend

SSH fingerprint have to be setup on the machine from which the class is instantiated

In [8]:
bm = BackendManager(server="login18-1.hpc.itc.rwth-aachen.de", uname="as641651")

In [9]:
bm.connect()

### Applications

####  1) Check if a file exists in the backend

In [7]:
bm.check_if_file_exists("~/pascal_voc.py")

True

#### 2) Copy file from backend

In [12]:
bm.copy_from_backend(backend_path="~/pascal_voc.py", local_path=".")

scp as641651@login18-1.hpc.itc.rwth-aachen.de:~/pascal_voc.py .
b''


0

#### 4) Build and Run commands

In [22]:
cmds = Commands()

In [16]:
cmd = cmds.build_cmd("python", "~/hello_world.py")
cmd

'cd ~; python hello_world.py '

In [17]:
stdout, ret_code = bm.run_cmd(cmd)

cd ~; python hello_world.py 


In [18]:
ret_code

0

In [19]:
stdout.readlines()

['Hello world\n']

#### 5) Build and execute commands on a batch system

A submit script should be available on the backend

In [51]:
cmd = cmds.submit_job_cmd("sbatch slurm_submit.sh", "python", "~/hello_world.py")
cmd

"cd ~; sbatch slurm_submit.sh python 'hello_world.py '"

In [57]:
stdout, ret_code = bm.run_cmd(cmd)

cd ~; sbatch slurm_submit.sh python 'hello_world.py '


In [58]:
stdout.readlines()

['Submitted batch job 28776139\n']

In [59]:
bm.check_slrum_status(jobname="Hello")# jobname should match the name in the submit script

['             JOBID PARTITION                           NAME     USER    STATE       TIME TIME_LIMI  NODES NODELIST(REASON)\n']


0

#### 6) Build custom commands

In [60]:
class MyCommands(Commands):
    def __init__(self, source="", submit_cmd=None):
        super().__init__(source)
        
        self.submit_cmd = submit_cmd
        
        self.interactive = True
        if self.submit_cmd:
            self.interactive = False
        
    def run_julia_cmd(self, script_path, args=""):
        if  self.interactive:
            cmd = self.build_cmd("julia", script_path, args)
        else:
            cmd = self.submit_job_cmd(self.submit_cmd, "julia", script_path, args)
        return cmd
    
    def run_python_cmd(self, script_path, args=""):
        if  self.interactive:
            cmd = self.build_cmd("python", script_path, args)
        else:
            cmd = self.submit_job_cmd(self.submit_cmd, "python", script_path, args)
        return cmd

In [61]:
mc= MyCommands()

In [62]:
mc.run_julia_cmd("~/PhD/runner.jl")

'cd ~/PhD; julia runner.jl '